# Notebook 04 — Bayesian Hierarchical Model (Per-Distance)
**RaceDayAI ML Prediction Engine (Plan 07)**

Separate NumPyro hierarchical models for 70.3 and 140.6. Partial pooling within each distance
over gender/age/cluster groups. Principled uncertainty quantification with credible intervals.

**Reads:** `athlete_race.csv`, `athlete_profile.csv`, `cluster_assignments.csv`
**Writes:** `bayesian_predictions_70.3.csv`, `bayesian_predictions_140.6.csv`, `bayesian_posterior_means.csv`

In [1]:
import pandas as pd
import numpy as np
import warnings
from pathlib import Path
from time import time
warnings.filterwarnings('ignore')

import jax
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS, Predictive

numpyro.set_host_device_count(1)  # CPU only
print(f"JAX devices: {jax.devices()}")
print(f"NumPyro version: {numpyro.__version__}")

BASE = Path('.').resolve().parent
CLEANED = BASE / 'data' / 'cleaned'

MODEL_DISTANCES = ['70.3', '140.6']

JAX devices: [CpuDevice(id=0)]
NumPyro version: 0.20.0


## 1. Load & Prepare Data

In [2]:
races = pd.read_csv(CLEANED / 'athlete_race.csv', low_memory=False)
profiles = pd.read_csv(CLEANED / 'athlete_profile.csv', low_memory=False)
clusters = pd.read_csv(CLEANED / 'cluster_assignments.csv', low_memory=False)

# Merge
pcols = ['athlete_hash', 'total_races', 'pb_total_sec', 'consistency_cv', 'improvement_slope']
pcols = [c for c in pcols if c in profiles.columns]
df = races.merge(profiles[pcols], on='athlete_hash', how='left')
ccols = ['athlete_hash', 'cluster_id']
ccols = [c for c in ccols if c in clusters.columns]
df = df.merge(clusters[ccols], on='athlete_hash', how='left')

# Filter: AG, valid times, 70.3 + 140.6
df = df[(df['is_pro'] != True) &
        df['event_distance'].isin(MODEL_DISTANCES) &
        df['total_sec'].notna() &
        (df['total_sec'] > 3600) & (df['total_sec'] < 61200)].copy()

for d in MODEL_DISTANCES:
    n = (df['event_distance'] == d).sum()
    print(f"  {d}: {n:,} records")
print(f"Total for Bayesian: {len(df):,}")

  70.3: 2,197,121 records
  140.6: 1,541,692 records
Total for Bayesian: 3,738,813


## 2. Define Hierarchical Model

Partial pooling: global mean + gender/age/cluster offsets + fitness/experience coefficients.
No distance parameter — we train separate models per distance.

In [3]:
def build_hierarchical_model(n_gender, n_age, n_cluster):
    def model(gender_idx, age_idx, cluster_idx, fitness, experience, y=None):
        # Global intercept
        mu_global = numpyro.sample('mu_global', dist.Normal(0.0, 1.0))

        # Group-level effects (partial pooling)
        sigma_gender = numpyro.sample('sigma_gender', dist.HalfNormal(0.5))
        mu_gender = numpyro.sample('mu_gender', dist.Normal(0, sigma_gender).expand([n_gender]))

        sigma_age = numpyro.sample('sigma_age', dist.HalfNormal(0.5))
        mu_age = numpyro.sample('mu_age', dist.Normal(0, sigma_age).expand([n_age]))

        sigma_cluster = numpyro.sample('sigma_cluster', dist.HalfNormal(0.5))
        mu_cluster = numpyro.sample('mu_cluster', dist.Normal(0, sigma_cluster).expand([n_cluster]))

        # Continuous coefficients
        beta_fitness = numpyro.sample('beta_fitness', dist.Normal(0.3, 0.2))
        beta_experience = numpyro.sample('beta_experience', dist.Normal(-0.1, 0.1))

        # Individual-level noise
        sigma = numpyro.sample('sigma', dist.HalfNormal(1.0))

        # Linear predictor
        mu = (mu_global
              + mu_gender[gender_idx]
              + mu_age[age_idx]
              + mu_cluster[cluster_idx]
              + beta_fitness * fitness
              + beta_experience * experience)

        # Likelihood
        numpyro.sample('obs', dist.Normal(mu, sigma), obs=y)

    return model

print("Model builder defined.")

Model builder defined.


## 3. Per-Distance MCMC

In [4]:
# Store results per distance
bayes_results = {}

for DIST in MODEL_DISTANCES:
    print(f"\n{'='*70}")
    print(f"  BAYESIAN MODEL: {DIST}")
    print(f"{'='*70}")

    dist_df = df[df['event_distance'] == DIST].copy()
    print(f"  Total records: {len(dist_df):,}")

    # Subsample for tractable MCMC
    N_SAMPLE = min(50000, len(dist_df))
    sample = dist_df.sample(N_SAMPLE, random_state=42).copy()
    print(f"  Subsampled to: {N_SAMPLE:,}")

    # Encode groups
    gender_map = {'M': 0, 'F': 1}
    sample['gender_idx'] = sample['gender'].map(gender_map).fillna(0).astype(int)
    N_GENDER = 2

    age_bands = sorted(sample['age_group'].dropna().unique())
    age_map = {a: i for i, a in enumerate(age_bands)}
    sample['age_idx'] = sample['age_group'].map(age_map).fillna(0).astype(int)
    N_AGE = len(age_bands)

    sample['cluster_id'] = sample['cluster_id'].fillna(-1).astype(int)
    cluster_vals = sorted(sample['cluster_id'].unique())
    cluster_map = {c: i for i, c in enumerate(cluster_vals)}
    sample['cluster_idx'] = sample['cluster_id'].map(cluster_map).astype(int)
    N_CLUSTER = len(cluster_vals)

    # Continuous features (standardize)
    dist_mean = sample['total_sec'].mean()
    dist_std = sample['total_sec'].std()
    sample['fitness'] = (sample['pb_total_sec'].fillna(dist_mean) - dist_mean) / dist_std
    sample['experience'] = np.log1p(sample['total_races'].fillna(0))
    exp_mean_s = sample['experience'].mean()
    exp_std_s = sample['experience'].std() + 1e-8
    sample['experience'] = (sample['experience'] - exp_mean_s) / exp_std_s

    # Standardized target
    sample['y_std'] = (sample['total_sec'] - dist_mean) / dist_std

    print(f"  Groups: {N_GENDER} genders, {N_AGE} age, {N_CLUSTER} clusters")
    print(f"  Target: mean={dist_mean:.0f}s  std={dist_std:.0f}s")

    # Build model
    model_fn = build_hierarchical_model(N_GENDER, N_AGE, N_CLUSTER)

    # JAX arrays
    data = {
        'gender_idx': jnp.array(sample['gender_idx'].values),
        'age_idx': jnp.array(sample['age_idx'].values),
        'cluster_idx': jnp.array(sample['cluster_idx'].values),
        'fitness': jnp.array(sample['fitness'].values),
        'experience': jnp.array(sample['experience'].values),
        'y': jnp.array(sample['y_std'].values),
    }

    # Run NUTS
    print(f"  Running NUTS: 1500 warmup + 1500 samples × 2 chains ...")
    t0 = time()
    kernel = NUTS(model_fn, target_accept_prob=0.85)
    mcmc = MCMC(kernel, num_warmup=1500, num_samples=1500, num_chains=2, progress_bar=True)
    mcmc.run(jax.random.PRNGKey(42 if DIST == '70.3' else 99), **data)
    elapsed = time() - t0
    print(f"  MCMC completed in {elapsed:.0f}s")
    mcmc.print_summary(exclude_deterministic=False)

    posterior = mcmc.get_samples()

    # Store everything needed for prediction
    bayes_results[DIST] = {
        'posterior': posterior,
        'dist_mean': dist_mean,
        'dist_std': dist_std,
        'gender_map': gender_map,
        'age_map': age_map,
        'cluster_map': cluster_map,
        'exp_mean': exp_mean_s,
        'exp_std': exp_std_s,
        'default_cluster_idx': sample['cluster_idx'].mode().values[0],
        'sample': sample,
        'data': data,
    }


  BAYESIAN MODEL: 70.3
  Total records: 2,197,121
  Subsampled to: 50,000
  Groups: 2 genders, 50 age, 9 clusters
  Target: mean=21193s  std=3286s
  Running NUTS: 1500 warmup + 1500 samples × 2 chains ...


sample: 100%|██████████| 3000/3000 [10:39<00:00,  4.69it/s, 511 steps of size 8.06e-03. acc. prob=0.95] 


  MCMC completed in 1324s

                       mean       std    median      5.0%     95.0%     n_eff     r_hat
  beta_experience      0.20      0.01      0.20      0.18      0.21   5559.05      1.00
     beta_fitness      0.48      0.00      0.48      0.47      0.48   8737.41      1.00
        mu_age[0]     -0.72      0.08     -0.72     -0.86     -0.59    367.75      1.00
        mu_age[1]     -0.30      0.23     -0.30     -0.69      0.07   3268.12      1.00
        mu_age[2]     -0.27      0.41     -0.26     -0.93      0.39   4776.04      1.00
        mu_age[3]     -0.77      0.29     -0.77     -1.25     -0.31   3413.79      1.00
        mu_age[4]      0.05      0.33      0.05     -0.45      0.59   4130.24      1.00
        mu_age[5]     -0.27      0.08     -0.27     -0.42     -0.15    367.21      1.00
        mu_age[6]      0.12      0.16      0.11     -0.15      0.35   1237.69      1.00
        mu_age[7]      0.14      0.41      0.14     -0.54      0.81   4860.90      1.00
     

sample: 100%|██████████| 3000/3000 [11:57<00:00,  4.18it/s, 511 steps of size 7.14e-03. acc. prob=0.92] 


  MCMC completed in 1454s

                       mean       std    median      5.0%     95.0%     n_eff     r_hat
  beta_experience      0.17      0.01      0.17      0.15      0.18   3521.83      1.00
     beta_fitness      0.45      0.00      0.45      0.44      0.45   4921.52      1.00
        mu_age[0]     -0.31      0.12     -0.31     -0.50     -0.10    257.46      1.00
        mu_age[1]     -0.53      0.49     -0.53     -1.30      0.26   2715.27      1.00
        mu_age[2]     -0.10      0.12     -0.10     -0.29      0.11    260.39      1.00
        mu_age[3]      0.42      0.30      0.41     -0.09      0.89   1474.37      1.00
        mu_age[4]      0.44      0.25      0.44      0.05      0.88   1038.10      1.00
        mu_age[5]     -0.68      0.15     -0.67     -0.93     -0.44    389.55      1.00
        mu_age[6]     -0.60      0.35     -0.59     -1.17     -0.02   1939.73      1.00
        mu_age[7]     -0.75      0.20     -0.74     -1.07     -0.40    790.72      1.00
     

## 4. Posterior Analysis (Per Distance)

In [5]:
for DIST in MODEL_DISTANCES:
    if DIST not in bayes_results:
        continue
    print(f"\n--- Posterior Analysis: {DIST} ---")
    r = bayes_results[DIST]
    posterior = r['posterior']
    y_std = r['dist_std']

    # Gender effects
    print("  Gender effects (in seconds):")
    for g, name in enumerate(['Male', 'Female']):
        vals = np.array(posterior['mu_gender'][:, g]) * y_std
        print(f"    {name}: mean={vals.mean():.0f}s  [{np.percentile(vals, 2.5):.0f}, {np.percentile(vals, 97.5):.0f}]")

    # Coefficients
    for param in ['beta_fitness', 'beta_experience', 'sigma']:
        vals = np.array(posterior[param])
        print(f"  {param}: mean={vals.mean():.3f}  std={vals.std():.3f}  "
              f"[{np.percentile(vals, 2.5):.3f}, {np.percentile(vals, 97.5):.3f}]")


--- Posterior Analysis: 70.3 ---
  Gender effects (in seconds):
    Male: mean=-482s  [-2445, 1681]
    Female: mean=716s  [-1246, 2890]
  beta_fitness: mean=0.478  std=0.004  [0.471, 0.486]
  beta_experience: mean=0.196  std=0.007  [0.182, 0.210]
  sigma: mean=0.786  std=0.002  [0.781, 0.790]

--- Posterior Analysis: 140.6 ---
  Gender effects (in seconds):
    Male: mean=-555s  [-4237, 4068]
    Female: mean=1549s  [-2136, 6148]
  beta_fitness: mean=0.448  std=0.003  [0.442, 0.453]
  beta_experience: mean=0.168  std=0.008  [0.153, 0.182]
  sigma: mean=0.687  std=0.002  [0.683, 0.691]


## 5. Posterior Predictive Checks (Per Distance)

In [6]:
from sklearn.metrics import mean_absolute_error, r2_score

for DIST in MODEL_DISTANCES:
    if DIST not in bayes_results:
        continue
    print(f"\n--- Posterior Predictive: {DIST} ---")
    r = bayes_results[DIST]
    posterior = r['posterior']
    data_noobs = {k: v for k, v in r['data'].items() if k != 'y'}

    model_fn = build_hierarchical_model(2, len(r['age_map']), len(r['cluster_map']))
    predictive = Predictive(model_fn, posterior)
    pred_samples = predictive(jax.random.PRNGKey(1), **data_noobs)

    pred_obs = np.array(pred_samples['obs']) * r['dist_std'] + r['dist_mean']
    pred_mean = pred_obs.mean(axis=0)
    pred_std = pred_obs.std(axis=0)
    pred_p05 = np.percentile(pred_obs, 5, axis=0)
    pred_p95 = np.percentile(pred_obs, 95, axis=0)

    y_actual = r['sample']['total_sec'].values
    mae = mean_absolute_error(y_actual, pred_mean)
    r2 = r2_score(y_actual, pred_mean)
    coverage_90 = ((y_actual >= pred_p05) & (y_actual <= pred_p95)).mean()

    print(f"  MAE: {mae/60:.1f} min ({mae:.0f} sec)")
    print(f"  R²:  {r2:.4f}")
    print(f"  90% CI coverage: {coverage_90:.3f} (target: 0.90)")
    print(f"  Mean prediction std: {pred_std.mean()/60:.1f} min")


--- Posterior Predictive: 70.3 ---
  MAE: 32.3 min (1936 sec)
  R²:  0.3828
  90% CI coverage: 0.914 (target: 0.90)
  Mean prediction std: 43.0 min

--- Posterior Predictive: 140.6 ---
  MAE: 55.3 min (3316 sec)
  R²:  0.5282
  90% CI coverage: 0.920 (target: 0.90)
  Mean prediction std: 72.3 min


## 6. Generate Predictions for Full Dataset (Per Distance)

In [7]:
for DIST in MODEL_DISTANCES:
    if DIST not in bayes_results:
        continue
    print(f"\n--- Full predictions: {DIST} ---")
    r = bayes_results[DIST]
    posterior = r['posterior']
    pm = {k: np.array(v).mean(axis=0) for k, v in posterior.items()}

    full_dist = df[df['event_distance'] == DIST].copy()

    # Encode
    full_dist['gender_idx'] = full_dist['gender'].map(r['gender_map']).fillna(0).astype(int)
    full_dist['age_idx'] = full_dist['age_group'].map(r['age_map']).fillna(0).astype(int)
    full_dist['cluster_id_clean'] = full_dist['cluster_id'].fillna(-1).astype(int)
    full_dist['cluster_idx'] = full_dist['cluster_id_clean'].map(r['cluster_map'])
    full_dist['cluster_idx'] = full_dist['cluster_idx'].fillna(r['default_cluster_idx']).astype(int)

    full_dist['fitness'] = (full_dist['pb_total_sec'].fillna(r['dist_mean']) - r['dist_mean']) / r['dist_std']
    full_dist['experience'] = np.log1p(full_dist['total_races'].fillna(0))
    full_dist['experience'] = (full_dist['experience'] - r['exp_mean']) / r['exp_std']

    # Posterior-mean prediction
    mu_pred = (pm['mu_global']
               + pm['mu_gender'][full_dist['gender_idx'].values]
               + pm['mu_age'][full_dist['age_idx'].values]
               + pm['mu_cluster'][full_dist['cluster_idx'].values]
               + pm['beta_fitness'] * full_dist['fitness'].values
               + pm['beta_experience'] * full_dist['experience'].values)

    full_dist['bayes_pred'] = mu_pred * r['dist_std'] + r['dist_mean']
    full_dist['bayes_std'] = float(pm['sigma']) * r['dist_std']

    print(f"  Predictions: {len(full_dist):,}")
    print(f"  Mean prediction: {full_dist['bayes_pred'].mean()/3600:.2f}h")
    print(f"  Mean uncertainty (1σ): {full_dist['bayes_std'].mean()/60:.1f}min")

    # Save
    out = full_dist[['athlete_hash', 'event_distance', 'event_year', 'total_sec',
                      'bayes_pred', 'bayes_std']].copy()
    fname = f'bayesian_predictions_{DIST}.csv'
    out.to_csv(CLEANED / fname, index=False)
    print(f"  → {fname}: {len(out):,}")


--- Full predictions: 70.3 ---
  Predictions: 2,197,121
  Mean prediction: 5.88h
  Mean uncertainty (1σ): 43.0min
  → bayesian_predictions_70.3.csv: 2,197,121

--- Full predictions: 140.6 ---
  Predictions: 1,541,692
  Mean prediction: 12.26h
  Mean uncertainty (1σ): 72.3min
  → bayesian_predictions_140.6.csv: 1,541,692


## 7. Save Posterior Summary

In [8]:
# Combined posterior parameter means
all_params = []
for DIST in MODEL_DISTANCES:
    if DIST not in bayes_results:
        continue
    posterior = bayes_results[DIST]['posterior']
    row = {'distance': DIST}
    for k, v in posterior.items():
        arr = np.array(v)
        if arr.ndim == 1:
            row[k] = arr.mean()
        else:
            for i in range(arr.shape[1]):
                row[f'{k}_{i}'] = arr[:, i].mean()
    all_params.append(row)

params_df = pd.DataFrame(all_params)
params_df.to_csv(CLEANED / 'bayesian_posterior_means.csv', index=False)
print(f"bayesian_posterior_means.csv: {len(params_df)} rows × {params_df.shape[1]} cols")

print("\n✅ BAYESIAN MODEL COMPLETE (per-distance)")

bayesian_posterior_means.csv: 2 rows × 69 cols

✅ BAYESIAN MODEL COMPLETE (per-distance)
